## 連立１次方程式の数値解法
$n$次正方行列 $A$, $n$項列ベクトル $b$ に対して，
連立一次方程式 $Ax = b$ の解は `A\b` で計算する．

In [60]:
A = [1 1 1; 1 2 3; 1 4 9]
b = [1, 2, 3]
A \ b

3-element Vector{Float64}:
 -0.5000000000000002
  2.0000000000000004
 -0.5000000000000002

念のため，行列式とランクを確かめておこう．

In [61]:
using LinearAlgebra
@show det(A) rank(A);

det(A) = 1.9999999999999996
rank(A) = 3


<div class="alert alert-warning"> 
Warning: 
連立１次方程式の解は inv(A)*b でも得られるが，A\b よりも効率が悪いので使われない．
</div>

## 係数行列がフルランクでない場合
簡約階段行列に変形して不定性も含めて求める．  
簡約階段行列の計算には`RowEchelon`パッケージの `rref()`を使うので，
あらかじめインストールしておく．  
（簡約階段の形状のことを英語で reduced echelon form という．）
```
pkg> add RowEchelon
```

次の係数行列に対して，$Bx = f$ を解くことを考える．

In [62]:
B = [1 3 -2 4;
    2 -1 3 0;
    8 3 5 8;
    7 -7 14 -4]

4×4 Matrix{Int64}:
 1   3  -2   4
 2  -1   3   0
 8   3   5   8
 7  -7  14  -4

In [63]:
@show rank(B) # ランクの確認

rank(B) = 2


2

In [64]:
# rref() で Bの簡約階段行列を計算する．
using RowEchelon
C = rref(B)

4×4 Matrix{Float64}:
 1.0  0.0   1.0  0.571429
 0.0  1.0  -1.0  1.14286
 0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0

`B` の kernelの基底を計算する．$ \vec{v} = (v_1, v_2, v_3, v_4) \in \ker B$は
$$
   C \vec{v} = \vec{0}
$$
と同値である．$(v_3,v_4) = (1,0), (0,1)$に対して それぞれ $(v_1, v_2)$
を求めれば，$\ker B$ の基底ベクトルが得られる．

したがって，次の方程式の解 $\vec{v} = (v_i), \vec{w} = (w_i)$が基底ベクトルになる．
$$
\begin{pmatrix}
1 & 0 & * & * \\
0 & 1 & * & * \\
0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 1 
\end{pmatrix}
\begin{pmatrix}v_1 & w_1 \\ v_2 & w_2 \\ v_3 & w_3 \\ v_4 & w_4 \end{pmatrix}
= 
\begin{pmatrix} 0 & 0 \\ 0 & 0 \\ 1 & 0 \\ 0 &  1\end{pmatrix}
$$

In [65]:
C[3, 3] = C[4, 4] = 1
@show C
v = C \ [0 0; 0 0; 1 0; 0 1]

C = [1.0 0.0 1.0 0.5714285714285714; 0.0 1.0 -1.0 1.1428571428571428; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]


4×2 Matrix{Float64}:
 -1.0  -0.571429
  1.0  -1.14286
  1.0   0.0
  0.0   1.0

In [66]:
# 確認
@show B * v[:, 1] B * v[:, 2];

B * v[:, 1] = [0.0, 0.0, 0.0, 0.0]
B * v[:, 2] = [0.0, 0.0, 0.0, 0.0]


### (i) $f = (6, 4, 24, 10)$ の場合
拡大係数行列 $(B | f)$のランクが 2，$(= \mathrm{rank} B)$なので，（不定性を含んだ）一般解が存在する．

In [67]:
f = [6, 4, 24, 10]  # = B*[1,1,1,1]
@show rank([B f])   # 拡大係数行列のランク
@show x = B \ f

rank([B f]) = 2
x = B \ f = [3.7142857142857144, 6.344131569286608e-17, -1.1428571428571428, 5.551115123125783e-17]


4-element Vector{Float64}:
  3.7142857142857144
  6.344131569286608e-17
 -1.1428571428571428
  5.551115123125783e-17

これに，先に求めた $\ker B$ を加えたものが一般解となる．

### (ii) $f = (1,2,3,4)$の場合
この場合は $\mathrm{rank} (B | f) = 3 > 2 = \mathrm{rank}$ となるので解は存在しない．

もし，これを知らずに解を計算するとどうなるか試してみる．

In [68]:
f = [1, 2, 3, 4]
@show rank([B f])  # 拡大係数行列のランク
@show x = B \ f    # 何らかの数値は出力される 
@show B * x
@show B * x == f

rank([B f]) = 3
x = B \ f = [-1.470563143631182e16, -2.058788401083655e16, 2.9411262872623645e15, 2.058788401083655e16]
B * x = [0.0, 6.0, 0.0, 32.0]
B * x == f = false


false